In [4]:
pip install PyPDF2


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 232.6/232.6 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
try:
    pdf=PdfReader('Corpus.pdf')
    result=''
    for i in range(len(pdf.pages)):
        result+=pdf.pages[i].extract_text()
except:
    print('PDF file doesn\'t exist')

In [14]:
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
      )
      
chunks = text_splitter.split_text(result)

In [21]:
hf_token='hf_token' #insert ur huggingface api key

api_url= """https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"""

headers = {"Authorization": f"Bearer {hf_token}"}
    
def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

user_question = 'Are dogs allowed at Jessup Cellars?'

question = query([user_question])
            
query_embeddings = torch.FloatTensor(question)
    
output=query(chunks)
    
output=torch.from_numpy(np.array(output)).to(torch.float)

In [22]:
result=util.semantic_search(query_embeddings, output,top_k=2)

final=[chunks[result[0][i]['corpus_id']] for i in range(len(result[0]))]

In [23]:
AI21_api_key ='api_key' #insert ur ai2i api key
url = "https://api.ai21.com/studio/v1/answer"
    
payload = {
                "context":' '.join(final),
                "question":user_question
          }
  
headers = {
                "accept": "application/json",
                "content-type": "application/json",
                "Authorization": f"Bearer {AI21_api_key}"
          }
    
response = requests.post(url, json=payload, headers=headers)
    
if response.json()['answerInContext']:
     print(response.json()['answer'])
else:
     print('The answer is not found  in the document ⚠️, kindly contact the business directly')

Yes, well-behaved dogs are allowed inside or on the outside patio at Jessup Cellars. Gluten-free dog treats and water dishes are available for dogs.
